In [ ]:
#!pip install networkx community

In [ ]:
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import community as community_louvain  
import ast
import random
import tarfile
import json

# Carica dati
edges = pd.read_csv('dataset/spoti/edges.csv')
nodes = pd.read_csv('dataset/spoti/nodes2.csv')
nodes_unique = nodes.drop_duplicates(subset=['spotify_id'], keep='first')


In [ ]:
# ========== PARAMETRI DI SELEZIONE ==========
TOPLIST = 2000
SELECTION_CRITERION = "degree"
POPULARITY_FIELD = "popularity"
COUNTRY_FILTER = "Italy"  # Opzioni: "Italy", "France", "Germany", "Spain", "United Kingdom", None (per tutti)
SEED = 42

np.random.seed(SEED)
random.seed(SEED)

In [ ]:
def is_from_country(nationality, country_filter=None):
    """
    Verifica se un artista appartiene a un paese specifico basandosi sulla nazionalità.
    
    Parametri:
    - nationality: stringa con il paese dell'artista (es. "Italy", "United States")
    - country_filter: nome del paese da filtrare (es. "Italy", "France") o None per nessun filtro
    
    Returns:
    - Boolean
    """
    if country_filter is None:
        return True
    
    if pd.isna(nationality):
        return False
    
    # Normalizza per il confronto (case-insensitive)
    nationality_lower = str(nationality).lower().strip()
    country_filter_lower = str(country_filter).lower().strip()
    
    return nationality_lower == country_filter_lower


In [ ]:
def parse_genres(genres_field):
    """Converte stringa rappresentante lista in lista Python"""
    if pd.isna(genres_field):
        return []
    if isinstance(genres_field, str):
        try:
            parsed = ast.literal_eval(genres_field)
            return parsed if isinstance(parsed, list) else []
        except:
            return []
    if isinstance(genres_field, list):
        return genres_field
    return []



In [ ]:
# ========== FUNZIONE DI SELEZIONE TOP NODI ==========
def select_top_nodes(graph, nodes_df, n=100, criterion="degree", popularity_field="popularity"):
    """
    Seleziona i top n nodi secondo diversi criteri di centralità.
    
    Parametri:
    - graph: grafo NetworkX
    - nodes_df: dataframe con attributi degli artisti
    - n: numero di nodi da selezionare
    - criterion: metrica di selezione ("degree", "popularity", "betweenness", "closeness", "eigenvector", "random")
    - popularity_field: nome colonna popolarità
    
    Returns:
    - Lista di nodi selezionati
    """
    
    if criterion == "degree":
        # Centralità di grado: numero di collaborazioni
        metric = dict(graph.degree())
        description = "numero di collaborazioni"
        
    elif criterion == "popularity":
        # Popolarità su Spotify
        metric = {}
        for node in graph.nodes():
            if node in nodes_df['spotify_id'].values:
                pop = nodes_df.loc[nodes_df['spotify_id'] == node, popularity_field].values
                metric[node] = pop[0] if len(pop) > 0 else 0
            else:
                metric[node] = 0
        description = "popolarità Spotify"
        
    elif criterion == "betweenness":
        # Centralità di intermediazione: broker tra comunità
        metric = nx.betweenness_centrality(graph)
        description = "centralità di intermediazione"
        
    elif criterion == "closeness":
        # Centralità di vicinanza: distanza media dagli altri
        metric = nx.closeness_centrality(graph)
        description = "centralità di vicinanza"
        
    elif criterion == "eigenvector":
        # Centralità di autovettore: connessione a nodi importanti
        try:
            metric = nx.eigenvector_centrality(graph, max_iter=1000)
            description = "centralità di autovettore"
        except:
            print("⚠️ Eigenvector centrality non convergente, uso degree")
            metric = dict(graph.degree())
            description = "grado (fallback)"
            
    elif criterion == "random":
        # Selezione casuale
        all_nodes = list(graph.nodes())
        random.shuffle(all_nodes)
        top_nodes = all_nodes[:n]
        
        print(f"\n[SELEZIONE TOP {n}]")
        print(f"Criterio: {criterion} (seed={SEED})")
        print(f"Primi 5 artisti selezionati casualmente:")
        for i, node in enumerate(top_nodes[:5], 1):
            name = graph.nodes[node].get('name', node)
            print(f"  {i}. {name}")
        
        return top_nodes
    else:
        raise ValueError(f"Criterio '{criterion}' non valido. Opzioni: degree, popularity, betweenness, closeness, eigenvector, random")
    
    # Ordina e seleziona top n (non per random, già gestito sopra)
    sorted_nodes = sorted(metric.items(), key=lambda x: x[1], reverse=True)
    top_nodes = [n for n, v in sorted_nodes[:n]]
    
    print(f"\n[SELEZIONE TOP {n}]")
    print(f"Criterio: {criterion} ({description})")
    print(f"Top 5 artisti selezionati:")
    for i, (node, value) in enumerate(sorted_nodes[:5], 1):
        name = graph.nodes[node].get('name', node)
        print(f"  {i}. {name} (valore: {value:.3f})")
    
    return top_nodes

In [ ]:
# ========== FILTRO PER NAZIONALITÀ ==========
# Filtra artisti per paese
nodes_country = nodes_unique[
    nodes_unique['nationality'].apply(lambda nat: is_from_country(nat, COUNTRY_FILTER))
]

print(f"✓ Artisti filtrati per nazionalità: {len(nodes_country)}")
if COUNTRY_FILTER:
    print(f"  Paese: {COUNTRY_FILTER}")
else:
    print(f"  Nessun filtro per nazionalità (globale)")

# Converti generi da stringa a lista
def parse_genres(genres_str):
    """Converte stringa di generi in lista"""
    if pd.isna(genres_str):
        return []
    if isinstance(genres_str, list):
        return genres_str
    try:
        return ast.literal_eval(genres_str)
    except:
        return []

nodes_country['genres'] = nodes_country['genres'].apply(parse_genres)
genres_with_data = sum(nodes_country['genres'].apply(len) > 0)
print(f"✓ Generi convertiti: {genres_with_data}/{len(nodes_country)} artisti con genere definito")

# Estrai IDs degli artisti del paese selezionato
country_ids = set(nodes_country['spotify_id'])
print(f"✓ Artisti {COUNTRY_FILTER if COUNTRY_FILTER else 'globali'} da analizzare: {len(country_ids)}")

# Filtra solo collaborazioni tra artisti dello stesso paese
print(f"\nFiltraggio collaborazioni...")
tt = edges[edges['id_0'].isin(country_ids) & edges['id_1'].isin(country_ids)]
print(f"✓ Collaborazioni interne: {len(tt)}")

# Creazione grafo completo
print(f"\nCreazione grafo...")
G_country = nx.Graph()
G_country.add_edges_from(tt[['id_0', 'id_1']].values)

# Aggiungi attributi dei nodi (nome, nazionalità, generi, ecc.)
attr_dict = nodes_country.set_index('spotify_id').to_dict('index')
nx.set_node_attributes(G_country, attr_dict)

print(f"\n{'='*60}")
print(f"GRAFO {'DEL PAESE: ' + COUNTRY_FILTER.upper() if COUNTRY_FILTER else 'GLOBALE'}")
print(f"{'='*60}")
print(f"Nodi (artisti): {G_country.number_of_nodes()}")
print(f"Archi (collaborazioni): {G_country.number_of_edges()}")

if G_country.number_of_nodes() > 0:
    density = nx.density(G_country)
    avg_degree = 2 * G_country.number_of_edges() / G_country.number_of_nodes()
    print(f"Densità: {density:.4f}")
    print(f"Grado medio: {avg_degree:.2f}")

# Visualizza info nazionalità nel grafo
print(f"\n{'='*60}")
print(f"DISTRIBUZIONE NAZIONALITÀ NEGLI ARTISTI")
print(f"{'='*60}")
nationality_dist = nodes_country['nationality'].value_counts()
print(nationality_dist.head(10))

In [ ]:
import ast
import pandas as pd

def parse_macro_genres(mg):
    if pd.isna(mg):
        return []
    if isinstance(mg, list):
        return mg
    try:
        return ast.literal_eval(mg)
    except:
        return []

nodes_country['macro_genres'] = nodes_country['macro_genres'].apply(parse_macro_genres)

with_macro = sum(nodes_country['macro_genres'].apply(len) > 0)
print(f"✓ Macro-generi convertiti: {with_macro}/{len(nodes_country)} artisti")
attr_dict = nodes_country.set_index('spotify_id').to_dict('index')
nx.set_node_attributes(G_country, attr_dict)


In [ ]:
'''
3 - Analisi sulla rete
	3.1 Community detection (Vedere se un artista collabora con artisti dello stesso genere)
		3.1.1 Louvain
		3.1.2 Analisi con 2.2.3
		3.1.3 Edge Betweenness Bart

	3.3 Degree distribution
'''
import community as community_louvain
from collections import Counter
import numpy as np
from networkx.algorithms.community import girvan_newman

print("\nEsecuzione Louvain community detection...")
partition_louvain = community_louvain.best_partition(G_country)

nx.set_node_attributes(G_country, partition_louvain, "community_louvain")

n_communities = len(set(partition_louvain.values()))
print(f"✓ Community trovate (Louvain): {n_communities}")


print("\nAnalisi omogeneità macro-generi per community (Louvain)...")

community_macro_stats = {}

for node, comm in partition_louvain.items():
    macro_genres = G_country.nodes[node].get("macro_genres", [])
    community_macro_stats.setdefault(comm, []).extend(macro_genres)

for comm, macros in community_macro_stats.items():
    if macros:
        most_common = Counter(macros).most_common(3)
        purity = most_common[0][1] / len(macros)

        print(f"Community {comm}:")
        print(f"  Macro-generi principali: {most_common}")
        print(f"  Purezza macro-genere dominante: {purity:.2f}")



print("\nEsecuzione Edge Betweenness (Girvan–Newman)...")

gn = girvan_newman(G_country)
first_split = next(gn)

communities_eb = [list(c) for c in first_split]
print(f"✓ Community trovate (Edge Betweenness): {len(communities_eb)}")

print("\nAnalisi macro-generi per community (Edge Betweenness)...")

for i, nodes_comm in enumerate(communities_eb):
    macros = []
    for node in nodes_comm:
        macros.extend(G_country.nodes[node].get("macro_genres", []))

    if macros:
        most_common = Counter(macros).most_common(3)
        purity = most_common[0][1] / len(macros)

        print(f"Community EB {i}:")
        print(f"  Macro-generi principali: {most_common}")
        print(f"  Purezza macro-genere dominante: {purity:.2f}")

print("\nComputing Degree Distribution...")

degrees = [deg for _, deg in G_country.degree()]

print("\nDegree Distribution:")
print(f"Minimum degree: {min(degrees)}")
print(f"Maximum degree: {max(degrees)}")
print(f"Average degree: {np.mean(degrees):.2f}")

# -------------------------
# Linear scale histogram
# -------------------------
plt.figure(figsize=(9,5))
plt.hist(degrees, bins=50, edgecolor='black', density=True)
plt.axvline(np.mean(degrees), linestyle='--', linewidth=1, label='Average degree')
plt.xlabel("Degree")
plt.ylabel("Probability density")
plt.title("Degree Distribution – Artist Collaborations")
plt.grid(alpha=0.3)
plt.legend()
plt.show()

# -------------------------
# Log-scale histogram (y-axis)
# -------------------------
plt.figure(figsize=(9,5))
plt.hist(degrees, bins=50, edgecolor='black', log=True, density=True)
plt.axvline(np.mean(degrees), linestyle='--', linewidth=1, label='Average degree')
plt.xlabel("Degree")
plt.ylabel("Probability density (log scale)")
plt.title("Degree Distribution (Log Scale)")
plt.grid(alpha=0.3)
plt.legend()
plt.show()



In [ ]:
#gio
print("\n" + "="*60)
print("9. GENERI ITALIANI CHE VANNO DI PIÙ ALL'ESTERO")
print("="*60)

# Filtra artisti italiani
italian_artists = nodes_unique[nodes_unique['nationality'] == 'Italy']
italian_ids = set(italian_artists['spotify_id'])

print(f"\nArtisti italiani totali: {len(italian_ids)}")

# Trova collaborazioni degli artisti italiani con artisti stranieri
italian_foreign_collabs = edges[
    ((edges['id_0'].isin(italian_ids)) & (~edges['id_1'].isin(italian_ids))) |
    ((edges['id_1'].isin(italian_ids)) & (~edges['id_0'].isin(italian_ids)))
]

print(f"Collaborazioni Italia-Estero: {len(italian_foreign_collabs)}")

# Identifica gli artisti italiani che collaborano all'estero
italian_artists_abroad = set()
for _, row in italian_foreign_collabs.iterrows():
    if row['id_0'] in italian_ids:
        italian_artists_abroad.add(row['id_0'])
    if row['id_1'] in italian_ids:
        italian_artists_abroad.add(row['id_1'])

print(f"Artisti italiani con collaborazioni estere: {len(italian_artists_abroad)}")

# Estrai i generi di questi artisti
italian_abroad_data = italian_artists[italian_artists['spotify_id'].isin(italian_artists_abroad)]

# Conta i generi
genre_counter = Counter()
for _, artist in italian_abroad_data.iterrows():
    genres = parse_genres(artist['genres'])
    for genre in genres:
        genre_counter[genre] += 1

# Calcola anche le collaborazioni per genere
genre_collab_count = Counter()
for _, artist in italian_abroad_data.iterrows():
    artist_id = artist['spotify_id']
    # Conta quante collaborazioni estere ha questo artista
    artist_collabs = italian_foreign_collabs[
        (italian_foreign_collabs['id_0'] == artist_id) |
        (italian_foreign_collabs['id_1'] == artist_id)
    ]
    num_collabs = len(artist_collabs)

    # Aggiungi alle statistiche del genere
    genres = parse_genres(artist['genres'])
    for genre in genres:
        genre_collab_count[genre] += num_collabs

print(f"\n{'='*60}")
print("TOP 15 GENERI ITALIANI PER PRESENZA ALL'ESTERO")
print(f"{'='*60}")
print(f"\n{'Genere':<35} {'Artisti':<10} {'Collab.':<10}")
print("-" * 60)

for genre, count in genre_counter.most_common(15):
    collab_count = genre_collab_count[genre]
    print(f"{genre:<35} {count:<10} {collab_count:<10}")

# Analisi per paese di destinazione
print(f"\n{'='*60}")
print("PAESI DI DESTINAZIONE DELLE COLLABORAZIONI ITALIANE")
print(f"{'='*60}")

foreign_countries = Counter()
for _, row in italian_foreign_collabs.iterrows():
    # Identifica l'artista straniero
    foreign_id = row['id_1'] if row['id_0'] in italian_ids else row['id_0']

    # Trova la nazionalità dell'artista straniero
    foreign_artist = nodes_unique[nodes_unique['spotify_id'] == foreign_id]
    if len(foreign_artist) > 0:
        nationality = foreign_artist.iloc[0]['nationality']
        if pd.notna(nationality):
            foreign_countries[nationality] += 1

print(f"\nTop 10 paesi per collaborazioni con artisti italiani:")
for country, count in foreign_countries.most_common(10):
    percentage = 100 * count / len(italian_foreign_collabs)
    print(f"  {country:<25} {count:>5} collaborazioni ({percentage:>5.1f}%)")

# Analisi dettagliata: genere per paese
print(f"\n{'='*60}")
print("GENERI ITALIANI PER PAESE DI DESTINAZIONE (Top 5 Paesi)")
print(f"{'='*60}")

top_countries = [country for country, _ in foreign_countries.most_common(5)]

for country in top_countries:
    print(f"\n{country.upper()}:")

    # Filtra collaborazioni con questo paese
    country_artist_ids = set(nodes_unique[
        nodes_unique['nationality'] == country
    ]['spotify_id'])

    country_collabs = italian_foreign_collabs[
        ((italian_foreign_collabs['id_0'].isin(italian_ids)) &
         (italian_foreign_collabs['id_1'].isin(country_artist_ids))) |
        ((italian_foreign_collabs['id_1'].isin(italian_ids)) &
         (italian_foreign_collabs['id_0'].isin(country_artist_ids)))
    ]

    # Identifica artisti italiani coinvolti
    italian_ids_with_country = set()
    for _, row in country_collabs.iterrows():
        if row['id_0'] in italian_ids:
            italian_ids_with_country.add(row['id_0'])
        if row['id_1'] in italian_ids:
            italian_ids_with_country.add(row['id_1'])

    # Conta generi
    country_genre_counter = Counter()
    for artist_id in italian_ids_with_country:
        artist = italian_artists[italian_artists['spotify_id'] == artist_id]
        if len(artist) > 0:
            genres = parse_genres(artist.iloc[0]['genres'])
            for genre in genres:
                country_genre_counter[genre] += 1

    # Mostra top 5 generi per questo paese
    print(f"  Top 5 generi italiani in {country}:")
    for genre, count in country_genre_counter.most_common(5):
        print(f"    - {genre}: {count} artisti")

# Artisti italiani più "internazionali"
print(f"\n{'='*60}")
print("TOP 10 ARTISTI ITALIANI PIÙ INTERNAZIONALI")
print(f"{'='*60}")

italian_artist_collab_count = Counter()
for artist_id in italian_artists_abroad:
    artist_collabs = italian_foreign_collabs[
        (italian_foreign_collabs['id_0'] == artist_id) |
        (italian_foreign_collabs['id_1'] == artist_id)
    ]
    italian_artist_collab_count[artist_id] = len(artist_collabs)

print(f"\n{'Artista':<30} {'Collab. Estere':<15} {'Generi Principali':<30}")
print("-" * 80)

for artist_id, collab_count in italian_artist_collab_count.most_common(10):
    artist = italian_artists[italian_artists['spotify_id'] == artist_id]
    if len(artist) > 0:
        name = artist.iloc[0]['name']
        genres = parse_genres(artist.iloc[0]['genres'])
        genre_str = ', '.join(genres[:2]) if len(genres) > 0 else 'N/A'
        print(f"{name:<30} {collab_count:<15} {genre_str:<30}")

print(f"\n{'='*60}")
print("CONCLUSIONI")
print(f"{'='*60}")

if len(genre_counter) > 0:
    top_genre = genre_counter.most_common(1)[0]
    print(f"\nGenere italiano più esportato: {top_genre[0]}")
    print(f"  → {top_genre[1]} artisti con collaborazioni estere")
    print(f"  → {genre_collab_count[top_genre[0]]} collaborazioni internazionali totali")

if len(foreign_countries) > 0:
    top_destination = foreign_countries.most_common(1)[0]
    print(f"\nPaese di destinazione principale: {top_destination[0]}")
    print(f"  → {top_destination[1]} collaborazioni")

percentage_abroad = 100 * len(italian_artists_abroad) / len(italian_ids)
print(f"\nPercentuale artisti italiani con presenza internazionale: {percentage_abroad:.1f}%")

In [ ]:
#gio
print("\n" + "="*60)
print("10. ARTISTI EMERGENTI VS AFFERMATI")
print("="*60)

# Definizione di artista emergente basata su popolarità e numero di collaborazioni
# Emergente: bassa popolarità MA attivo (ha collaborazioni)
# Affermato: alta popolarità E molte collaborazioni

# Verifica disponibilità colonna popularity
if 'popularity' not in nodes_unique.columns:
    print("⚠️  Colonna 'popularity' non disponibile nel dataset")
else:
    # Calcola numero di collaborazioni per artista
    collab_count = pd.concat([
        edges['id_0'].value_counts(),
        edges['id_1'].value_counts()
    ]).groupby(level=0).sum()

    # Crea DataFrame con popolarità e collaborazioni
    artist_metrics = nodes_unique[['spotify_id', 'name', 'popularity', 'nationality']].copy()
    artist_metrics['num_collabs'] = artist_metrics['spotify_id'].map(collab_count).fillna(0).astype(int)

    # Rimuovi artisti senza collaborazioni
    artist_metrics = artist_metrics[artist_metrics['num_collabs'] > 0]

    print(f"\nArtisti con almeno 1 collaborazione: {len(artist_metrics)}")

    # Definisci soglie per classificazione
    # Emergente: popolarità < 40 E collaborazioni >= 3
    # Affermato: popolarità >= 60 E collaborazioni >= 5
    # Intermedio: il resto

    EMERGING_POP_THRESHOLD = 40
    EMERGING_COLLAB_MIN = 3
    ESTABLISHED_POP_THRESHOLD = 60
    ESTABLISHED_COLLAB_MIN = 5

    def classify_artist(row):
        if row['popularity'] < EMERGING_POP_THRESHOLD and row['num_collabs'] >= EMERGING_COLLAB_MIN:
            return 'Emergente'
        elif row['popularity'] >= ESTABLISHED_POP_THRESHOLD and row['num_collabs'] >= ESTABLISHED_COLLAB_MIN:
            return 'Affermato'
        else:
            return 'Intermedio'

    artist_metrics['category'] = artist_metrics.apply(classify_artist, axis=1)

    # Statistiche generali
    print(f"\n{'='*60}")
    print("CLASSIFICAZIONE ARTISTI")
    print(f"{'='*60}")

    category_counts = artist_metrics['category'].value_counts()
    total = len(artist_metrics)

    for category in ['Emergente', 'Intermedio', 'Affermato']:
        if category in category_counts.index:
            count = category_counts[category]
            pct = 100 * count / total
            print(f"\n{category}: {count} artisti ({pct:.1f}%)")

    # Statistiche dettagliate per categoria
    print(f"\n{'='*60}")
    print("STATISTICHE PER CATEGORIA")
    print(f"{'='*60}")

    stats_by_category = artist_metrics.groupby('category').agg({
        'popularity': ['mean', 'median', 'min', 'max'],
        'num_collabs': ['mean', 'median', 'min', 'max']
    }).round(1)

    print("\nPOPOLARITÀ:")
    print(f"{'Categoria':<15} {'Media':<10} {'Mediana':<10} {'Min':<10} {'Max':<10}")
    print("-" * 60)
    for category in ['Emergente', 'Intermedio', 'Affermato']:
        if category in stats_by_category.index:
            row = stats_by_category.loc[category]
            print(f"{category:<15} {row[('popularity', 'mean')]:<10} {row[('popularity', 'median')]:<10} "
                  f"{row[('popularity', 'min')]:<10} {row[('popularity', 'max')]:<10}")

    print("\nCOLLABORAZIONI:")
    print(f"{'Categoria':<15} {'Media':<10} {'Mediana':<10} {'Min':<10} {'Max':<10}")
    print("-" * 60)
    for category in ['Emergente', 'Intermedio', 'Affermato']:
        if category in stats_by_category.index:
            row = stats_by_category.loc[category]
            print(f"{category:<15} {row[('num_collabs', 'mean')]:<10} {row[('num_collabs', 'median')]:<10} "
                  f"{row[('num_collabs', 'min')]:<10} {row[('num_collabs', 'max')]:<10}")

    # Top artisti per categoria
    print(f"\n{'='*60}")
    print("TOP 10 ARTISTI EMERGENTI (per numero di collaborazioni)")
    print(f"{'='*60}")

    emerging = artist_metrics[artist_metrics['category'] == 'Emergente'].sort_values('num_collabs', ascending=False)

    print(f"\n{'Artista':<30} {'Popolarità':<12} {'Collaborazioni':<15} {'Nazionalità':<20}")
    print("-" * 80)
    for _, artist in emerging.head(10).iterrows():
        nat = artist['nationality'] if pd.notna(artist['nationality']) else 'N/A'
        print(f"{artist['name']:<30} {artist['popularity']:<12} {artist['num_collabs']:<15} {nat:<20}")

    print(f"\n{'='*60}")
    print("TOP 10 ARTISTI AFFERMATI (per numero di collaborazioni)")
    print(f"{'='*60}")

    established = artist_metrics[artist_metrics['category'] == 'Affermato'].sort_values('num_collabs', ascending=False)

    print(f"\n{'Artista':<30} {'Popolarità':<12} {'Collaborazioni':<15} {'Nazionalità':<20}")
    print("-" * 80)
    for _, artist in established.head(10).iterrows():
        nat = artist['nationality'] if pd.notna(artist['nationality']) else 'N/A'
        print(f"{artist['name']:<30} {artist['popularity']:<12} {artist['num_collabs']:<15} {nat:<20}")

    # Distribuzione per nazionalità
    print(f"\n{'='*60}")
    print("DISTRIBUZIONE PER NAZIONALITÀ (Top 10 Paesi)")
    print(f"{'='*60}")

    # Filtra solo artisti con nazionalità definita
    artists_with_nat = artist_metrics[artist_metrics['nationality'].notna()]

    nat_category = artists_with_nat.groupby(['nationality', 'category']).size().unstack(fill_value=0)
    nat_category['Total'] = nat_category.sum(axis=1)

    # Filtra paesi con almeno 50 artisti
    nat_category = nat_category[nat_category['Total'] >= 50].sort_values('Total', ascending=False)

    # Calcola percentuali
    for col in ['Emergente', 'Intermedio', 'Affermato']:
        if col in nat_category.columns:
            nat_category[f'{col}_pct'] = 100 * nat_category[col] / nat_category['Total']

    print(f"\n{'Paese':<20} {'Totale':<10} {'Emergenti':<12} {'Intermedi':<12} {'Affermati':<12}")
    print("-" * 70)
    for country, row in nat_category.head(10).iterrows():
        emg_pct = row.get('Emergente_pct', 0)
        int_pct = row.get('Intermedio_pct', 0)
        est_pct = row.get('Affermato_pct', 0)
        print(f"{country:<20} {int(row['Total']):<10} {emg_pct:>5.1f}%  {int_pct:>5.1f}%  {est_pct:>5.1f}%")

    # Focus Italia
    if 'Italy' in nat_category.index:
        print(f"\n{'='*60}")
        print("FOCUS ITALIA")
        print(f"{'='*60}")

        italy_row = nat_category.loc['Italy']
        italian_artists = artists_with_nat[artists_with_nat['nationality'] == 'Italy']

        print(f"\nArtisti italiani totali: {int(italy_row['Total'])}")

        for category in ['Emergente', 'Intermedio', 'Affermato']:
            if category in nat_category.columns:
                count = italy_row[category]
                pct = italy_row[f'{category}_pct']
                print(f"  {category}: {int(count)} ({pct:.1f}%)")

        # Top emergenti italiani
        print(f"\nTop 5 artisti emergenti italiani:")
        italian_emerging = italian_artists[italian_artists['category'] == 'Emergente'].sort_values('num_collabs', ascending=False)

        for i, (_, artist) in enumerate(italian_emerging.head(5).iterrows(), 1):
            print(f"  {i}. {artist['name']}: {artist['num_collabs']} collab., popolarità {artist['popularity']}")

        # Top affermati italiani
        print(f"\nTop 5 artisti affermati italiani:")
        italian_established = italian_artists[italian_artists['category'] == 'Affermato'].sort_values('num_collabs', ascending=False)

        for i, (_, artist) in enumerate(italian_established.head(5).iterrows(), 1):
            print(f"  {i}. {artist['name']}: {artist['num_collabs']} collab., popolarità {artist['popularity']}")

    # Analisi pattern di collaborazione
    print(f"\n{'='*60}")
    print("PATTERN DI COLLABORAZIONE: EMERGENTI VS AFFERMATI")
    print(f"{'='*60}")

    # Prepara mapping per analisi
    category_map = artist_metrics.set_index('spotify_id')['category'].to_dict()

    # Aggiungi categoria a edges
    edges_cat = edges.copy()
    edges_cat['cat_0'] = edges_cat['id_0'].map(category_map)
    edges_cat['cat_1'] = edges_cat['id_1'].map(category_map)

    # Filtra solo collaborazioni dove entrambi hanno categoria
    edges_cat = edges_cat.dropna(subset=['cat_0', 'cat_1'])

    # Conta tipo di collaborazioni
    collab_types = pd.crosstab(edges_cat['cat_0'], edges_cat['cat_1'])

    print("\nMatrice di collaborazioni:")
    print(collab_types)

    # Calcola percentuali
    print("\n% di collaborazioni per tipo:")
    total_collabs = edges_cat.shape[0]

    for cat1 in ['Emergente', 'Intermedio', 'Affermato']:
        for cat2 in ['Emergente', 'Intermedio', 'Affermato']:
            if cat1 in collab_types.index and cat2 in collab_types.columns:
                count = collab_types.loc[cat1, cat2]
                pct = 100 * count / total_collabs
                print(f"  {cat1} ↔ {cat2}: {int(count)} ({pct:.1f}%)")

    # Tendenza: emergenti collaborano con affermati?
    if 'Emergente' in collab_types.index:
        emg_total = collab_types.loc['Emergente'].sum()
        emg_with_established = collab_types.loc['Emergente', 'Affermato'] if 'Affermato' in collab_types.columns else 0
        emg_with_emerging = collab_types.loc['Emergente', 'Emergente']

        print(f"\n{'='*60}")
        print("STRATEGIA ARTISTI EMERGENTI")
        print(f"{'='*60}")

        pct_established = 100 * emg_with_established / emg_total
        pct_emerging = 100 * emg_with_emerging / emg_total

        print(f"\nGli artisti emergenti collaborano:")
        print(f"  - Con altri emergenti: {int(emg_with_emerging)} ({pct_emerging:.1f}%)")
        print(f"  - Con artisti affermati: {int(emg_with_established)} ({pct_established:.1f}%)")

        if pct_established > pct_emerging:
            print("\n→ Gli emergenti tendono a collaborare DI PIÙ con artisti affermati")
            print("  (strategia di 'boost' tramite artisti famosi)")
        else:
            print("\n→ Gli emergenti tendono a collaborare DI PIÙ tra loro")
            print("  (networking orizzontale tra peer)")

    print(f"\n{'='*60}")
    print("CONCLUSIONI")
    print(f"{'='*60}")

    if 'Emergente' in category_counts.index and 'Affermato' in category_counts.index:
        emg_count = category_counts['Emergente']
        est_count = category_counts['Affermato']

        print(f"""
✓ Dataset contiene {emg_count} artisti emergenti e {est_count} artisti affermati

✓ Gli emergenti hanno in media {emerging['num_collabs'].mean():.1f} collaborazioni
  (vs {established['num_collabs'].mean():.1f} per gli affermati)

✓ La popolarità media degli emergenti è {emerging['popularity'].mean():.1f}
  (vs {established['popularity'].mean():.1f} per gli affermati)
        """)

In [ ]:
# ========== CREAZIONE SOTTOGRAFO ANALISI ==========
# Seleziona i top artisti
top_nodes = select_top_nodes(G_country, nodes_country, n=TOPLIST, criterion=SELECTION_CRITERION)

# Crea sottografo con solo i top artisti e le loro collaborazioni
G_top = G_country.subgraph(top_nodes).copy()

print(f"\n[GRAFO TOP {TOPLIST}]")
print(f"Nodi: {G_top.number_of_nodes()}")
print(f"Archi: {G_top.number_of_edges()}")

# ========== STATISTICHE GENERALIZZATE ==========
print(f"\n[SUMMARY PARAMETRI]")
print(f"  Paese: {COUNTRY_FILTER.upper() if COUNTRY_FILTER else 'GLOBALE'}")
print(f"  Criterio selezione: {SELECTION_CRITERION}")
print(f"  Top: {TOPLIST} artisti")
print(f"  Grafo completo: {G_country.number_of_nodes()} nodi, {G_country.number_of_edges()} archi")
print(f"  Grafo analisi: {G_top.number_of_nodes()} nodi, {G_top.number_of_edges()} archi")

In [ ]:
print("\n" + "="*60)
print("1. PROPRIETÀ GLOBALI DELLA RETE")
print("="*60)

# Densità: rapporto tra archi esistenti e possibili
density = nx.density(G_top)
print(f"\nDensità: {density:.4f}")
print(f"  → La rete è {'molto' if density > 0.1 else 'poco'} connessa")

# Componenti connesse
components = list(nx.connected_components(G_top))
print(f"\nComponenti connesse: {len(components)}")
print(f"  → Dimensione componente principale: {len(max(components, key=len))} nodi")

# Lavora sulla componente gigante
G_main = G_top.subgraph(max(components, key=len)).copy()

# Diametro e cammino medio (solo se connesso)
if nx.is_connected(G_main):
    diameter = nx.diameter(G_main)
    avg_path = nx.average_shortest_path_length(G_main)
    print(f"\nDiametro: {diameter}")
    print(f"  → Massima distanza tra due artisti: {diameter} passaggi")
    print(f"\nCammino medio: {avg_path:.3f}")
    print(f"  → Distanza media tra artisti: {avg_path:.3f} collaborazioni")
else:
    print("\nGrafo non completamente connesso, diametro non calcolabile")

# Coefficiente di clustering
clustering_coeff = nx.average_clustering(G_main)
print(f"\nCoefficient di clustering medio: {clustering_coeff:.4f}")
print(f"  → I collaboratori di un artista {'spesso' if clustering_coeff > 0.3 else 'raramente'} collaborano tra loro")


In [ ]:
print("\n" + "="*60)
print("2. DISTRIBUZIONE DEI GRADI")
print("="*60)

# Calcola gradi
degrees = [G_main.degree(n) for n in G_main.nodes()]
degree_count = Counter(degrees)

# Statistiche
avg_degree = np.mean(degrees)
median_degree = np.median(degrees)
max_degree = max(degrees)

print(f"\nGrado medio: {avg_degree:.2f}")
print(f"Grado mediano: {median_degree:.0f}")
print(f"Grado massimo: {max_degree}")

# Trova artisti più connessi
top_degree = sorted(G_main.degree(), key=lambda x: x[1], reverse=True)[:10]
print(f"\nTop 10 artisti per numero di collaborazioni:")
for i, (node, deg) in enumerate(top_degree, 1):
    name = G_main.nodes[node].get('name', node)
    print(f"  {i}. {name}: {deg} collaborazioni")

# Test distribuzione power-law
print(f"\nLa distribuzione {'segue' if max_degree > 5*avg_degree else 'non segue'} un pattern scale-free")
print(f"  → {'Pochi hub dominano' if max_degree > 5*avg_degree else 'Rete più omogenea'}")


In [ ]:
print("\n" + "="*60)
print("3. MISURE DI CENTRALITÀ")
print("="*60)

# Calcola diverse centralità
degree_cent = nx.degree_centrality(G_main)
betweenness_cent = nx.betweenness_centrality(G_main)
closeness_cent = nx.closeness_centrality(G_main)

try:
    eigenvector_cent = nx.eigenvector_centrality(G_main, max_iter=1000)
except:
    eigenvector_cent = {n: 0 for n in G_main.nodes()}

# Crea dataframe comparativo
centrality_df = pd.DataFrame({
    'artist': [G_main.nodes[n].get('name', n) for n in G_main.nodes()],
    'degree': [degree_cent[n] for n in G_main.nodes()],
    'betweenness': [betweenness_cent[n] for n in G_main.nodes()],
    'closeness': [closeness_cent[n] for n in G_main.nodes()],
    'eigenvector': [eigenvector_cent[n] for n in G_main.nodes()]
})

print("\n[DEGREE CENTRALITY] - Artisti più connessi:")
top_deg = centrality_df.nlargest(5, 'degree')
for idx, row in top_deg.iterrows():
    print(f"  {row['artist']}: {row['degree']:.4f}")

print("\n[BETWEENNESS CENTRALITY] - Artisti bridge tra comunità:")
top_bet = centrality_df.nlargest(5, 'betweenness')
for idx, row in top_bet.iterrows():
    print(f"  {row['artist']}: {row['betweenness']:.4f}")

print("\n[CLOSENESS CENTRALITY] - Artisti centrali nella rete:")
top_clo = centrality_df.nlargest(5, 'closeness')
for idx, row in top_clo.iterrows():
    print(f"  {row['artist']}: {row['closeness']:.4f}")

print("\n[EIGENVECTOR CENTRALITY] - Artisti connessi ad altri importanti:")
top_eig = centrality_df.nlargest(5, 'eigenvector')
for idx, row in top_eig.iterrows():
    print(f"  {row['artist']}: {row['eigenvector']:.4f}")


In [ ]:
print("\n" + "="*60)
print("4. RILEVAMENTO COMUNITÀ")
print("="*60)

# Algoritmo Louvain per community detection
partition = community_louvain.best_partition(G_main)

# Aggiungi community come attributo
nx.set_node_attributes(G_main, partition, 'community')

# Analisi comunità
num_communities = len(set(partition.values()))
print(f"\nNumero di comunità rilevate: {num_communities}")

# Dimensione comunità
community_sizes = Counter(partition.values())
print(f"\nDistribuzione dimensioni comunità:")
for comm_id, size in sorted(community_sizes.items(), key=lambda x: x[1], reverse=True)[:10]:
    print(f"  Comunità {comm_id}: {size} artisti")

# Modularity
modularity = community_louvain.modularity(partition, G_main)
print(f"\nModularità: {modularity:.4f}")
print(f"  → {'Forte' if modularity > 0.4 else 'Moderata' if modularity > 0.3 else 'Debole'} struttura a comunità")

# Artisti per comunità (top 3 comunità più grandi)
print(f"\nArtisti principali per comunità (top 3 comunità):")
for comm_id, size in sorted(community_sizes.items(), key=lambda x: x[1], reverse=True)[:3]:
    print(f"\n  Comunità {comm_id} ({size} artisti):")
    comm_nodes = [n for n, c in partition.items() if c == comm_id]
    comm_degrees = [(n, G_main.degree(n)) for n in comm_nodes]
    top_in_comm = sorted(comm_degrees, key=lambda x: x[1], reverse=True)[:5]
    for node, deg in top_in_comm:
        name = G_main.nodes[node].get('name', node)
        print(f"    - {name} ({deg} collab.)")


In [ ]:
print("\n" + "="*60)
print("5. BRIDGE ANALYSIS - CONNETTORI TRA COMUNITÀ")
print("="*60)

# Identifica edge betweenness
edge_betweenness = nx.edge_betweenness_centrality(G_main)

# Top bridge edges
top_bridges = sorted(edge_betweenness.items(), key=lambda x: x[1], reverse=True)[:10]

print("\nTop 10 collaborazioni-bridge:")
for i, ((u, v), score) in enumerate(top_bridges, 1):
    name_u = G_main.nodes[u].get('name', u)
    name_v = G_main.nodes[v].get('name', v)
    comm_u = partition[u]
    comm_v = partition[v]
    is_bridge = "✓" if comm_u != comm_v else "✗"
    print(f"  {i}. {name_u} ↔ {name_v} (score: {score:.4f}) [Bridge: {is_bridge}]")

# Constraint (Burt's structural holes)
constraint = nx.constraint(G_main)
low_constraint = sorted(constraint.items(), key=lambda x: x[1])[:10]

print("\nArtisti con accesso a structural holes (basso constraint):")
for i, (node, const) in enumerate(low_constraint, 1):
    name = G_main.nodes[node].get('name', node)
    print(f"  {i}. {name} (constraint: {const:.4f})")


In [ ]:
print("\n" + "="*60)
print("6. CLIQUES E GRUPPI COESI")
print("="*60)

# Trova cliques massimali
cliques = list(nx.find_cliques(G_main))
clique_sizes = [len(c) for c in cliques]

print(f"\nNumero totale di cliques: {len(cliques)}")
print(f"Dimensione massima clique: {max(clique_sizes)}")
print(f"Dimensione media clique: {np.mean(clique_sizes):.2f}")

# Top 5 cliques più grandi
largest_cliques = sorted(cliques, key=len, reverse=True)[:5]
print(f"\nTop 5 cliques più grandi:")
for i, clique in enumerate(largest_cliques, 1):
    print(f"\n  Clique {i} ({len(clique)} artisti):")
    for node in clique[:10]:  # Mostra max 10
        name = G_main.nodes[node].get('name', node)
        print(f"    - {name}")

# K-core decomposition
k_cores = nx.core_number(G_main)
max_k = max(k_cores.values())

print(f"\nK-core massimo: {max_k}")
print(f"  → Esiste un nucleo di artisti con almeno {max_k} connessioni reciproche")

# Artisti nel k-core massimo
max_core_nodes = [n for n, k in k_cores.items() if k == max_k]
print(f"\nArtisti nel {max_k}-core ({len(max_core_nodes)} artisti):")
for node in sorted(max_core_nodes, key=lambda n: G_main.degree(n), reverse=True)[:10]:
    name = G_main.nodes[node].get('name', node)
    degree = G_main.degree(node)
    print(f"  - {name} ({degree} collab.)")


In [ ]:
print("\n" + "="*60)
print("7. PROPRIETÀ SMALL-WORLD")
print("="*60)

# Usa il sottografo principale della tua selezione
# (rinomina G_main → G_top o usa qualunque variabile rappresenti il grafo in analisi)
G_analysis = G_top  # Puoi facilmente cambiarlo in G_country o altro in base ai tuoi settings

# Calcola coefficienti su G_analysis
C_real = nx.average_clustering(G_analysis)
if nx.is_connected(G_analysis):
    L_real = nx.average_shortest_path_length(G_analysis)
else:
    L_real = None

# Parametri base del grafo per generare confronto random
n = G_analysis.number_of_nodes()
m = G_analysis.number_of_edges()
if n > 1:
    p = 2 * m / (n * (n - 1))
else:
    p = 0

# Genera grafo random Erdős–Rényi con stesso numero di nodi e archi attesi
G_random = nx.erdos_renyi_graph(n, p, seed=SEED)
C_random = nx.average_clustering(G_random)

if nx.is_connected(G_random):
    L_random = nx.average_shortest_path_length(G_random)
else:
    # Se non connesso, usa componente gigante
    largest_cc = max(nx.connected_components(G_random), key=len)
    G_random_main = G_random.subgraph(largest_cc)
    L_random = nx.average_shortest_path_length(G_random_main)

# Output generalizzato
print(f"\nRete musicale selezionata:")
print(f"  Clustering: {C_real:.4f}")
if L_real:
    print(f"  Cammino medio: {L_real:.4f}")

print(f"\nGrafo random equivalente:")
print(f"  Clustering: {C_random:.4f}")
print(f"  Cammino medio: {L_random:.4f}")

# Test small-world
if L_real:
    sigma = (C_real / C_random) / (L_real / L_random)
    print(f"\nSmall-world coefficient (σ): {sigma:.4f}")
    if sigma > 1:
        print(f"  ✓ La rete selezionata ha proprietà SMALL-WORLD")
        print(f"    → Alto clustering locale + brevi distanze globali")
    else:
        print(f"  ✗ La rete selezionata non mostra forti proprietà small-world")
else:
    print("\nCammino medio non calcolabile (rete non connessa)")


In [ ]:
print("\n" + "="*60)
print("8. ANALISI PER GENERE MUSICALE")
print("="*60)

# Verifica se esiste campo genere
if 'genres' in G_main.nodes[list(G_main.nodes())[0]]:
    
    # Estrai generi principali
    genre_dict = {}
    for node in G_main.nodes():
        genres = G_main.nodes[node].get('genres', [])
        if isinstance(genres, list) and len(genres) > 0:
            # Prendi primo genere
            genre_dict[node] = genres[0]
        else:
            genre_dict[node] = 'Unknown'
    
    # Distribuzione generi
    genre_count = Counter(genre_dict.values())
    print(f"\nDistribuzione generi (top 10):")
    for genre, count in genre_count.most_common(10):
        print(f"  {genre}: {count} artisti")
    
    # Assortatività per genere
    nx.set_node_attributes(G_main, genre_dict, 'genre')
    
    # Collaborazioni intra vs inter-genere
    intra_genre = 0
    inter_genre = 0
    for u, v in G_main.edges():
        if genre_dict[u] == genre_dict[v]:
            intra_genre += 1
        else:
            inter_genre += 1
    
    total_edges = intra_genre + inter_genre
    print(f"\nCollaborazioni intra-genere: {intra_genre} ({100*intra_genre/total_edges:.1f}%)")
    print(f"Collaborazioni inter-genere: {inter_genre} ({100*inter_genre/total_edges:.1f}%)")
    
    if intra_genre > inter_genre:
        print("  → Gli artisti tendono a collaborare dentro lo stesso genere")
    else:
        print("  → Forte cross-pollination tra generi diversi")
    
else:
    print("\nAttributo 'genres' non disponibile nel dataset")


In [ ]:
import os

# --- Prepara la rete per export ---
# Converti tipi non supportati (list, dict, tuple) in stringa:
for n, data in G_top.nodes(data=True):
    for k, v in data.items():
        if isinstance(v, (list, dict, tuple)):
            G_top.nodes[n][k] = str(v)

# Prepara nome directory e file:
country = COUNTRY_FILTER if COUNTRY_FILTER else "all"
crit = SELECTION_CRITERION
topn = TOPLIST
export_dir = "exports_gexf"
os.makedirs(export_dir, exist_ok=True)
filename = f"{country}_{crit}_top{topn}.gexf"
filepath = os.path.join(export_dir, filename)

# Exporta il grafo
nx.write_gexf(G_top, filepath)

print(f"\n✓ Esportazione completata!")
print(f"  File GEXF salvato in: {filepath}")
print(f"  → Aprilo in Gephi per analisi/visualizzazione.")


In [ ]:
# ============================================================================
# ANALISI CONNETTIVITÀ DELLA RETE
# ============================================================================

def analyze_network_connectivity(G):
    """
    Analizza la connettività della rete e identifica componenti connesse.
    
    Parametri:
    - G: grafo NetworkX
    
    Returns:
    - Dizionario con statistiche di connettività
    """
    
    print("="*70)
    print("ANALISI CONNETTIVITÀ DELLA RETE")
    print("="*70)
    
    # Verifica base
    print(f"\nNodi totali: {G.number_of_nodes()}")
    print(f"Archi totali: {G.number_of_edges()}")
    
    # Controlla se il grafo è connesso
    is_connected = nx.is_connected(G)
    print(f"\n🔍 Grafo completamente connesso: {is_connected}")
    
    if not is_connected:
        print("⚠️ La rete NON è completamente connessa!")
        print("   Esistono componenti isolate o non raggiungibili.")
    else:
        print("✓ La rete è completamente connessa!")
        print("   Tutti i nodi sono raggiungibili da tutti gli altri.")
    
    # ========== COMPONENTI CONNESSE ==========
    print(f"\n{'-'*70}")
    print("COMPONENTI CONNESSE")
    print(f"{'-'*70}")
    
    # Trova tutte le componenti connesse
    components = list(nx.connected_components(G))
    num_components = len(components)
    
    print(f"Numero di componenti connesse: {num_components}")
    
    if num_components == 1:
        print("✓ C'è una sola componente: la rete è connessa")
    else:
        print(f"❌ Ci sono {num_components} componenti separate\n")
        
        # Ordina per dimensione
        components_sorted = sorted(components, key=len, reverse=True)
        
        print("Distribuzione delle componenti:")
        for i, comp in enumerate(components_sorted[:10], 1):  # Mostra top 10
            percentage = 100 * len(comp) / G.number_of_nodes()
            print(f"  Componente {i}: {len(comp)} nodi ({percentage:.1f}%)")
        
        if num_components > 10:
            other_count = sum(len(c) for c in components_sorted[10:])
            print(f"  ... e {num_components - 10} altre componenti minori ({100*other_count/G.number_of_nodes():.1f}%)")
    
    # ========== DIAMETRO E DISTANZE ==========
    print(f"\n{'-'*70}")
    print("DISTANZE E DIAMETRO")
    print(f"{'-'*70}")
    
    if is_connected:
        # Se il grafo è connesso, calcola il diametro
        diameter = nx.diameter(G)
        print(f"Diametro della rete: {diameter}")
        print(f"  (massima distanza tra due nodi qualsiasi)")
        
        # Eccentricità dei nodi
        eccentricity = nx.eccentricity(G)
        avg_eccentricity = sum(eccentricity.values()) / len(eccentricity)
        print(f"Eccentricità media: {avg_eccentricity:.2f}")
        
        # Raggio
        radius = nx.radius(G)
        print(f"Raggio della rete: {radius}")
        print(f"  (minima eccentricità)")
        
    else:
        # Se non connesso, analizza per componente
        print("Analisi per componente connessa:\n")
        for i, comp in enumerate(components_sorted[:5], 1):
            G_sub = G.subgraph(comp)
            if G_sub.number_of_nodes() > 1:
                diam = nx.diameter(G_sub)
                print(f"  Componente {i} (nodi: {len(comp)}): diametro = {diam}")
    
    # ========== LUNGHEZZA MEDIA DEI PATH ==========
    print(f"\n{'-'*70}")
    print("LUNGHEZZA MEDIA DEI PATH (Average Path Length)")
    print(f"{'-'*70}")
    
    if is_connected:
        avg_path_length = nx.average_shortest_path_length(G)
        print(f"Lunghezza media dei path: {avg_path_length:.2f}")
        print(f"  (in media, due artisti sono distanti {avg_path_length:.2f} collaborazioni)")
    else:
        print("Calcolo per componente connessa:\n")
        for i, comp in enumerate(components_sorted[:5], 1):
            G_sub = G.subgraph(comp)
            if G_sub.number_of_nodes() > 1:
                avg_path = nx.average_shortest_path_length(G_sub)
                print(f"  Componente {i}: {avg_path:.2f}")
    
    # ========== COEFFICIENTE DI CLUSTERING ==========
    print(f"\n{'-'*70}")
    print("CLUSTERING (Densità Locale)")
    print(f"{'-'*70}")
    
    clustering_coeff = nx.average_clustering(G)
    print(f"Coefficiente di clustering medio: {clustering_coeff:.4f}")
    print(f"  (misura quanto i vicini di un nodo sono connessi tra loro)")
    
    # ========== ANALISI DEI NODI ISOLATI ==========
    print(f"\n{'-'*70}")
    print("NODI ISOLATI E PERIFERICI")
    print(f"{'-'*70}")
    
    isolated_nodes = list(nx.isolates(G))
    print(f"Nodi isolati (grado = 0): {len(isolated_nodes)}")
    
    if len(isolated_nodes) > 0:
        print("  ⚠️ Questi artisti non hanno collaborazioni")
        # Mostra i nomi di alcuni artisti isolati
        if 'name' in G.nodes[list(G.nodes())[0]]:
            isolated_names = [G.nodes[node].get('name', node) for node in isolated_nodes[:5]]
            print(f"  Esempi: {', '.join(isolated_names)}")
    
    # Nodi con basso grado (periferici)
    degrees = dict(G.degree())
    low_degree_nodes = [node for node, degree in degrees.items() if degree <= 2]
    print(f"Nodi periferici (grado ≤ 2): {len(low_degree_nodes)}")
    
    # ========== STATISTICHE FINALI ==========
    print(f"\n{'-'*70}")
    print("STATISTICHE FINALI")
    print(f"{'-'*70}")
    
    degree_values = list(dict(G.degree()).values())
    print(f"Grado minimo: {min(degree_values)}")
    print(f"Grado massimo: {max(degree_values)}")
    print(f"Grado medio: {sum(degree_values) / len(degree_values):.2f}")
    
    density = nx.density(G)
    print(f"Densità della rete: {density:.4f}")
    print(f"  (0 = grafo vuoto, 1 = grafo completo)")
    
    # Ritorna un dizionario con i risultati
    return {
        'is_connected': is_connected,
        'num_components': num_components,
        'num_nodes': G.number_of_nodes(),
        'num_edges': G.number_of_edges(),
        'density': density,
        'num_isolated': len(isolated_nodes),
        'clustering_coeff': clustering_coeff
    }


# ============================================================================
# ESECUZIONE
# ============================================================================

# Analizza la rete principale
results = analyze_network_connectivity(G_main)

print("\n" + "="*70)
print("INTERPRETAZIONE")
print("="*70)

if results['is_connected']:
    print("✓ La rete è COMPLETAMENTE CONNESSA")
    print("  Ogni artista può raggiungere ogni altro artista attraverso collaborazioni")
else:
    print("❌ La rete NON è completamente connessa")
    print(f"  Ci sono {results['num_components']} componenti separate")
    print(f"  Solo il {100*results['num_components']/results['num_nodes']:.1f}% dei nodi è nella componente principale")

if results['num_isolated'] > 0:
    print(f"\n⚠️ ATTENZIONE: {results['num_isolated']} artisti sono isolati (nessuna collaborazione)")

print(f"\nDensità: {results['density']:.4f}")
if results['density'] < 0.01:
    print("  → Rete molto sparsa (poche collaborazioni)")
elif results['density'] < 0.1:
    print("  → Rete moderatamente sparsa")
else:
    print("  → Rete densa (molte collaborazioni)")


In [ ]:
print("ANALISI: ARTISTI POPOLARI IN NAZIONI DIVERSE DALLA PROPRIA NAZIONALITÀ")
print("="*60)

import pandas as pd
import ast
from collections import Counter
import numpy as np

# Funzione per parsare il formato ['at (44)', 'de (111)', 'lu (22)']
def parse_chart_hits_list(chart_hits_str):
    """Parsa stringhe come "['at (44)', 'de (111)', 'lu (22)']" in dizionario"""
    if pd.isna(chart_hits_str):
        return {}
    
    result = {}
    
    try:
        if isinstance(chart_hits_str, list):
            items = chart_hits_str
        elif isinstance(chart_hits_str, str):
            cleaned = chart_hits_str.strip("[]").replace("'", "").replace('"', "")
            items = [item.strip() for item in cleaned.split(",") if item.strip()]
        else:
            return {}
        
        for item in items:
            if '(' in item and ')' in item:
                country_part = item.split('(')[0].strip()
                number_part = item.split('(')[1].split(')')[0].strip()
                
                if country_part and number_part.isdigit():
                    result[country_part.upper()] = int(number_part)
    
    except Exception as e:
        pass
    
    return result

# Dizionario per mappare codici paese
country_codes_map = {
    'US': 'United States', 'USA': 'United States', 'U.S.': 'United States',
    'UK': 'United Kingdom', 'GB': 'United Kingdom', 'U.K.': 'United Kingdom',
    'DE': 'Germany', 'GER': 'Germany',
    'FR': 'France', 'FRA': 'France',
    'IT': 'Italy', 'ITA': 'Italy',
    'ES': 'Spain', 'ESP': 'Spain',
    'NL': 'Netherlands', 'NED': 'Netherlands',
    'SE': 'Sweden', 'SWE': 'Sweden',
    'NO': 'Norway', 'NOR': 'Norway',
    'DK': 'Denmark', 'DEN': 'Denmark',
    'FI': 'Finland', 'FIN': 'Finland',
    'AT': 'Austria', 'AUT': 'Austria',
    'CH': 'Switzerland', 'SUI': 'Switzerland',
    'BE': 'Belgium', 'BEL': 'Belgium',
    'PT': 'Portugal', 'POR': 'Portugal',
    'IE': 'Ireland', 'IRL': 'Ireland',
    'AU': 'Australia', 'AUS': 'Australia',
    'CA': 'Canada', 'CAN': 'Canada',
    'JP': 'Japan', 'JPN': 'Japan',
    'KR': 'South Korea', 'KOR': 'South Korea',
    'CN': 'China', 'CHN': 'China',
    'BR': 'Brazil', 'BRA': 'Brazil',
    'MX': 'Mexico', 'MEX': 'Mexico',
    'AR': 'Argentina', 'ARG': 'Argentina',
    'RU': 'Russia', 'RUS': 'Russia',
    'PL': 'Poland', 'POL': 'Poland',
    'CZ': 'Czech Republic', 'CZE': 'Czech Republic',
    'HU': 'Hungary', 'HUN': 'Hungary',
    'GR': 'Greece', 'GRE': 'Greece',
    'TR': 'Turkey', 'TUR': 'Turkey',
    'IN': 'India', 'IND': 'India'
}

def get_country_name(code):
    """Ottiene il nome completo del paese dal codice"""
    if pd.isna(code):
        return "Unknown"
    code_upper = str(code).upper()
    return country_codes_map.get(code_upper, code_upper)

def safe_str(value, default=""):
    """Converte un valore in stringa in modo sicuro"""
    if pd.isna(value):
        return default
    try:
        return str(value)
    except:
        return default

# Parsa chart_hits per tutti gli artisti
print("\nParsing chart_hits...")
nodes_unique['chart_hits_parsed'] = nodes_unique['chart_hits'].apply(parse_chart_hits_list)

# Analisi per artista
print("Analizzando artisti...")
artist_stats = []

for idx, row in nodes_unique.iterrows():
    artist_id = row['spotify_id']
    artist_name = safe_str(row['name'], "Unknown Artist")
    artist_nationality = safe_str(row['nationality'], "UNKNOWN").upper()
    chart_hits = row['chart_hits_parsed']
    
    # Verifica se abbiamo popularity
    if 'popularity' in row:
        popularity = float(row['popularity']) if pd.notna(row['popularity']) else 0.0
    else:
        popularity = 0.0
    
    if not chart_hits:
        continue
    
    # Calcola totali
    total_hits = sum(chart_hits.values())
    
    # Trova hits in paesi diversi dalla propria nazionalità
    foreign_hits = {}
    domestic_hits = {}
    
    for country_code, hit_count in chart_hits.items():
        country_code_norm = safe_str(country_code).upper()
        
        if country_code_norm == artist_nationality:
            domestic_hits[country_code_norm] = hit_count
        else:
            foreign_hits[country_code_norm] = hit_count
    
    # Calcola metriche
    foreign_hits_total = sum(foreign_hits.values())
    domestic_hits_total = sum(domestic_hits.values())
    
    if foreign_hits:  # Solo artisti con successo all'estero
        percent_foreign = (foreign_hits_total / total_hits * 100) if total_hits > 0 else 0
        percent_domestic = (domestic_hits_total / total_hits * 100) if total_hits > 0 else 0
        
        # Ordina paesi esteri per hits
        foreign_countries_sorted = sorted(foreign_hits.items(), key=lambda x: x[1], reverse=True)
        
        artist_stats.append({
            'artist_id': artist_id,
            'name': artist_name,
            'nationality': artist_nationality,
            'nationality_full': get_country_name(artist_nationality),
            'popularity': popularity,
            'total_hits': total_hits,
            'num_foreign_countries': len(foreign_hits),
            'foreign_hits_total': foreign_hits_total,
            'domestic_hits_total': domestic_hits_total,
            'percent_foreign': percent_foreign,
            'percent_domestic': percent_domestic,
            'foreign_hits_by_country': foreign_hits,
            'domestic_hits_by_country': domestic_hits,
            'foreign_countries': list(foreign_hits.keys()),
            'foreign_countries_full': [(get_country_name(c), h) for c, h in foreign_countries_sorted],
            'top_foreign_country': foreign_countries_sorted[0] if foreign_countries_sorted else None,
            'all_countries': list(chart_hits.keys())
        })

print(f"\nAnalisi completata!")
print(f"Artisti totali nel dataset: {len(nodes_unique)}")
print(f"Artisti con chart_hits: {len(nodes_unique[nodes_unique['chart_hits_parsed'].apply(len) > 0])}")
print(f"Artisti con successo all'estero: {len(artist_stats)}")

# SALVATAGGIO SU FILE
print("\n" + "="*60)
print("SALVATAGGIO SU FILE: foreign_popularity.txt")
print("="*60)

with open('foreign_popularity.txt', 'w', encoding='utf-8') as f:
    # INTESTAZIONE
    f.write("="*80 + "\n")
    f.write("ANALISI: ARTISTI POPOLARI IN NAZIONI DIVERSE DALLA PROPRIA NAZIONALITÀ\n")
    f.write("="*80 + "\n\n")
    
    f.write("📊 STATISTICHE GLOBALI:\n")
    f.write(f"• Artisti totali nel dataset: {len(nodes_unique)}\n")
    f.write(f"• Artisti con chart_hits: {len(nodes_unique[nodes_unique['chart_hits_parsed'].apply(len) > 0])}\n")
    f.write(f"• Artisti con successo all'estero: {len(artist_stats)}\n")
    f.write(f"• Percentuale artisti internazionali: {100*len(artist_stats)/len(nodes_unique):.1f}%\n\n")
    
    if len(artist_stats) > 0:
        # Crea DataFrame per calcoli aggregati
        intl_df = pd.DataFrame(artist_stats)
        
        # 1. STATISTICHE MEDIE
        f.write("📈 METRICHE MEDIE (artisti con successo estero):\n")
        f.write(f"• Popolarità media: {intl_df['popularity'].mean():.1f}\n")
        f.write(f"• Hits totali medi: {intl_df['total_hits'].mean():.1f}\n")
        f.write(f"• Paesi esteri medi per artista: {intl_df['num_foreign_countries'].mean():.1f}\n")
        f.write(f"• Hits estere medie per artista: {intl_df['foreign_hits_total'].mean():.1f}\n")
        f.write(f"• % successo all'estero media: {intl_df['percent_foreign'].mean():.1f}%\n")
        f.write(f"• % successo in patria media: {intl_df['percent_domestic'].mean():.1f}%\n\n")
        
        # 2. TOP ARTISTI PER POPOLARITÀ
        f.write("="*80 + "\n")
        f.write("TOP 50 ARTISTI PIÙ POPOLARI CON SUCCESSO INTERNAZIONALE\n")
        f.write("(Ordinati per popolarità decrescente)\n")
        f.write("="*80 + "\n\n")
        
        top_popular = intl_df.sort_values('popularity', ascending=False).head(50)
        
        for i, (_, artist) in enumerate(top_popular.iterrows(), 1):
            f.write(f"{i:3d}. {artist['name']}\n")
            f.write(f"     Nazionalità: {artist['nationality_full']} ({artist['nationality']})\n")
            f.write(f"     Popolarità: {artist['popularity']:.0f}/100\n")
            f.write(f"     Hits totali: {artist['total_hits']}\n")
            f.write(f"     Paesi esteri: {artist['num_foreign_countries']}\n")
            f.write(f"     Hits estere: {artist['foreign_hits_total']} ({artist['percent_foreign']:.1f}% del totale)\n")
            f.write(f"     Hits in patria: {artist['domestic_hits_total']} ({artist['percent_domestic']:.1f}% del totale)\n")
            
            # Top 5 paesi esteri
            if artist['foreign_countries_full']:
                f.write(f"     Top 5 paesi esteri:\n")
                for country_name, hits in artist['foreign_countries_full'][:5]:
                    f.write(f"       • {country_name}: {hits} hits\n")
            
            f.write("\n")
        
        # 3. ARTISTI PIÙ INTERNAZIONALI (più paesi)
        f.write("="*80 + "\n")
        f.write("TOP 50 ARTISTI PIÙ INTERNAZIONALI\n")
        f.write("(Più paesi esteri diversi di successo)\n")
        f.write("="*80 + "\n\n")
        
        top_international = intl_df.sort_values(['num_foreign_countries', 'foreign_hits_total', 'popularity'], 
                                              ascending=[False, False, False]).head(50)
        
        for i, (_, artist) in enumerate(top_international.iterrows(), 1):
            f.write(f"{i:3d}. {artist['name']}\n")
            f.write(f"     Nazionalità: {artist['nationality_full']} ({artist['nationality']})\n")
            f.write(f"     Paesi esteri: {artist['num_foreign_countries']}\n")
            f.write(f"     Popolarità: {artist['popularity']:.0f}/100\n")
            f.write(f"     Hits estere totali: {artist['foreign_hits_total']}\n")
            
            # Lista completa paesi esteri
            if artist['foreign_countries_full']:
                f.write(f"     Paesi esteri (ordinati per hits):\n")
                for country_name, hits in artist['foreign_countries_full'][:10]:  # Max 10 paesi
                    f.write(f"       • {country_name}: {hits} hits\n")
                if len(artist['foreign_countries_full']) > 10:
                    f.write(f"       • ... e altri {len(artist['foreign_countries_full']) - 10} paesi\n")
            
            f.write("\n")
        
        # 4. ARTISTI CON LA MAGGIOR % DI SUCCESSO ALL'ESTERO
        f.write("="*80 + "\n")
        f.write("TOP 50 ARTISTI CON LA MAGGIOR % DI SUCCESSO ALL'ESTERO\n")
        f.write("(Più 'esportabili' - minimo 50 hits totali)\n")
        f.write("="*80 + "\n\n")
        
        significant = intl_df[intl_df['total_hits'] >= 50]
        if len(significant) > 0:
            top_exportable = significant.sort_values('percent_foreign', ascending=False).head(50)
            
            for i, (_, artist) in enumerate(top_exportable.iterrows(), 1):
                f.write(f"{i:3d}. {artist['name']}\n")
                f.write(f"     Nazionalità: {artist['nationality_full']} ({artist['nationality']})\n")
                f.write(f"     % successo all'estero: {artist['percent_foreign']:.1f}%\n")
                f.write(f"     % successo in patria: {artist['percent_domestic']:.1f}%\n")
                f.write(f"     Hits totali: {artist['total_hits']}\n")
                f.write(f"     Hits estere: {artist['foreign_hits_total']}\n")
                f.write(f"     Popolarità: {artist['popularity']:.0f}/100\n")
                
                if artist['top_foreign_country']:
                    country_code, hits = artist['top_foreign_country']
                    country_name = get_country_name(country_code)
                    f.write(f"     Paese estero principale: {country_name} ({hits} hits)\n")
                
                f.write("\n")
        
        # 5. ARTISTI PER NAZIONALITÀ
        f.write("="*80 + "\n")
        f.write("STATISTICHE PER NAZIONALITÀ\n")
        f.write("="*80 + "\n\n")
        
        # Raggruppa per nazionalità
        nationality_stats = intl_df.groupby('nationality_full').agg({
            'artist_id': 'count',
            'popularity': 'mean',
            'total_hits': 'mean',
            'num_foreign_countries': 'mean',
            'foreign_hits_total': 'mean',
            'percent_foreign': 'mean'
        }).round(2)
        
        nationality_stats = nationality_stats.rename(columns={
            'artist_id': 'Num_Artisti',
            'popularity': 'Popolarità_Media',
            'total_hits': 'Hits_Totali_Media',
            'num_foreign_countries': 'Paesi_Estero_Media',
            'foreign_hits_total': 'Hits_Estere_Media',
            'percent_foreign': '%_Estere_Media'
        }).sort_values('Num_Artisti', ascending=False)
        
        f.write(f"{'Nazionalità':<25} {'Artisti':<10} {'Pop. Media':<12} {'Hits Tot. Media':<15} {'Paesi Est. Media':<15} {'% Estere Media':<15}\n")
        f.write("-" * 100 + "\n")
        
        for nationality, row in nationality_stats.iterrows():
            f.write(f"{nationality:<25} {row['Num_Artisti']:<10} {row['Popolarità_Media']:<12.1f} "
                   f"{row['Hits_Totali_Media']:<15.1f} {row['Paesi_Estero_Media']:<15.1f} {row['%_Estere_Media']:<15.1f}%\n")
        
        f.write("\n")
        
        # 6. PAESI DI DESTINAZIONE PREFERITI
        f.write("="*80 + "\n")
        f.write("PAESI DI DESTINAZIONE PREFERITI\n")
        f.write("(Dove vanno gli artisti stranieri)\n")
        f.write("="*80 + "\n\n")
        
        destination_stats = {}
        for _, artist in intl_df.iterrows():
            for country_code, hits in artist['foreign_hits_by_country'].items():
                country_name = get_country_name(country_code)
                if country_name not in destination_stats:
                    destination_stats[country_name] = {
                        'artists': set(),
                        'total_hits': 0,
                        'nationalities': set()
                    }
                
                destination_stats[country_name]['artists'].add(artist['artist_id'])
                destination_stats[country_name]['total_hits'] += hits
                destination_stats[country_name]['nationalities'].add(artist['nationality_full'])
        
        # Converti per ordinamento
        destination_list = []
        for country_name, data in destination_stats.items():
            destination_list.append({
                'country': country_name,
                'num_artists': len(data['artists']),
                'total_hits': data['total_hits'],
                'num_nationalities': len(data['nationalities']),
                'avg_hits_per_artist': data['total_hits'] / len(data['artists']) if data['artists'] else 0
            })
        
        destination_df = pd.DataFrame(destination_list)
        destination_df = destination_df.sort_values(['num_artists', 'total_hits'], ascending=[False, False])
        
        f.write(f"{'Paese Dest.':<20} {'Artisti':<12} {'Hits Totali':<15} {'Nazionalità Diverse':<20} {'Hits/Artista':<15}\n")
        f.write("-" * 85 + "\n")
        
        for _, row in destination_df.head(30).iterrows():
            f.write(f"{row['country']:<20} {row['num_artists']:<12} {row['total_hits']:<15} "
                   f"{row['num_nationalities']:<20} {row['avg_hits_per_artist']:<15.1f}\n")
        
        f.write("\n")
        
        # 7. DETTAGLIO COMPLETO DI TUTTI GLI ARTISTI
        f.write("="*80 + "\n")
        f.write("DETTAGLIO COMPLETO DI TUTTI GLI ARTISTI\n")
        f.write(f"(In ordine alfabetico - {len(artist_stats)} artisti totali)\n")
        f.write("="*80 + "\n\n")
        
        # Funzione di ordinamento sicura
        def get_sort_key(artist):
            name = artist['name']
            if isinstance(name, (int, float)):
                return str(name).lower()
            return name.lower() if isinstance(name, str) else ""
        
        # Ordina alfabeticamente per nome
        sorted_artists = sorted(artist_stats, key=get_sort_key)
        
        for i, artist in enumerate(sorted_artists, 1):
            f.write(f"ARTISTA {i}/{len(sorted_artists)}: {artist['name']}\n")
            f.write(f"{'-'*60}\n")
            f.write(f"ID: {artist['artist_id']}\n")
            f.write(f"Nazionalità: {artist['nationality_full']} (codice: {artist['nationality']})\n")
            f.write(f"Popolarità Spotify: {artist['popularity']:.0f}/100\n")
            f.write(f"\n📊 STATISTICHE CHART HITS:\n")
            f.write(f"• Hits totali: {artist['total_hits']}\n")
            f.write(f"• Hits in patria: {artist['domestic_hits_total']} ({artist['percent_domestic']:.1f}%)\n")
            f.write(f"• Hits all'estero: {artist['foreign_hits_total']} ({artist['percent_foreign']:.1f}%)\n")
            f.write(f"• Paesi esteri con successo: {artist['num_foreign_countries']}\n")
            
            f.write(f"\n🌍 PAESI CON SUCCESSO:\n")
            
            # Hits in patria
            if artist['domestic_hits_by_country']:
                for country_code, hits in artist['domestic_hits_by_country'].items():
                    country_name = get_country_name(country_code)
                    f.write(f"  • {country_name} (patria): {hits} hits\n")
            
            # Hits all'estero (ordinati)
            if artist['foreign_countries_full']:
                f.write(f"\n  Paesi esteri (ordinati per hits):\n")
                for country_name, hits in artist['foreign_countries_full']:
                    f.write(f"    • {country_name}: {hits} hits\n")
            else:
                f.write(f"  Nessun successo all'estero registrato\n")
            
            f.write(f"\n🎯 TOP PAESE ESTERO: ")
            if artist['top_foreign_country']:
                country_code, hits = artist['top_foreign_country']
                country_name = get_country_name(country_code)
                f.write(f"{country_name} con {hits} hits\n")
            else:
                f.write(f"Nessuno\n")
            
            f.write(f"\n{'='*60}\n\n")
        
        # 8. RECORD E PRIMATI
        f.write("="*80 + "\n")
        f.write("RECORD E PRIMATI\n")
        f.write("="*80 + "\n\n")
        
        if len(intl_df) > 0:
            # Artista più popolare
            most_popular = intl_df.loc[intl_df['popularity'].idxmax()]
            f.write(f"🎤 ARTISTA PIÙ POPOLARE:\n")
            f.write(f"  • Nome: {most_popular['name']}\n")
            f.write(f"  • Nazionalità: {most_popular['nationality_full']}\n")
            f.write(f"  • Popolarità: {most_popular['popularity']:.0f}/100\n")
            f.write(f"  • Paesi esteri: {most_popular['num_foreign_countries']}\n")
            f.write(f"  • % successo estero: {most_popular['percent_foreign']:.1f}%\n\n")
            
            # Artista più internazionale (più paesi)
            most_international = intl_df.loc[intl_df['num_foreign_countries'].idxmax()]
            f.write(f"🌎 ARTISTA PIÙ INTERNAZIONALE:\n")
            f.write(f"  • Nome: {most_international['name']}\n")
            f.write(f"  • Nazionalità: {most_international['nationality_full']}\n")
            f.write(f"  • Paesi esteri: {most_international['num_foreign_countries']}\n")
            f.write(f"  • Lista paesi: {', '.join(most_international['foreign_countries'])}\n\n")
            
            # Artista con più hits estere
            most_foreign_hits = intl_df.loc[intl_df['foreign_hits_total'].idxmax()]
            f.write(f"🔥 ARTISTA CON PIÙ HITS ESTERE:\n")
            f.write(f"  • Nome: {most_foreign_hits['name']}\n")
            f.write(f"  • Nazionalità: {most_foreign_hits['nationality_full']}\n")
            f.write(f"  • Hits estere: {most_foreign_hits['foreign_hits_total']}\n")
            f.write(f"  • % successo estero: {most_foreign_hits['percent_foreign']:.1f}%\n\n")
            
            # Artista più "esportabile" (% più alta)
            significant_artists = intl_df[intl_df['total_hits'] >= 50]
            if len(significant_artists) > 0:
                most_exportable = significant_artists.loc[significant_artists['percent_foreign'].idxmax()]
                f.write(f"🚀 ARTISTA PIÙ 'ESPORTABILE':\n")
                f.write(f"  • Nome: {most_exportable['name']}\n")
                f.write(f"  • Nazionalità: {most_exportable['nationality_full']}\n")
                f.write(f"  • % successo estero: {most_exportable['percent_foreign']:.1f}%\n")
                f.write(f"  • Hits totali: {most_exportable['total_hits']}\n")
                if most_exportable['top_foreign_country']:
                    country_code, hits = most_exportable['top_foreign_country']
                    country_name = get_country_name(country_code)
                    f.write(f"  • Top paese estero: {country_name} ({hits} hits)\n")
        
        # 9. MAPPATURA CODICI PAESE TROVATI
        f.write("="*80 + "\n")
        f.write("MAPPATURA CODICI PAESE\n")
        f.write("="*80 + "\n\n")
        
        all_codes = set()
        for artist in artist_stats:
            all_codes.update(artist['all_countries'])
        
        f.write(f"Codici paese trovati nel dataset ({len(all_codes)} totali):\n")
        for code in sorted(all_codes):
            full_name = get_country_name(code)
            f.write(f"  • {code}: {full_name}\n")
        
        # 10. INFORMAZIONI TECNICHE
        f.write("="*80 + "\n")
        f.write("INFORMAZIONI TECNICHE\n")
        f.write("="*80 + "\n\n")
        
        
        from datetime import datetime
        f.write(f"Data di generazione: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"Numero totale di artisti analizzati: {len(nodes_unique)}\n")
        f.write(f"Artisti con dati chart_hits: {len(nodes_unique[nodes_unique['chart_hits_parsed'].apply(len) > 0])}\n")
        f.write(f"Artisti con successo all'estero: {len(artist_stats)}\n")
        f.write(f"Soglia per successo: almeno 1 hit in paese straniero\n")
        f.write(f"Metodo di calcolo: hits in paesi diversi dalla nazionalità dell'artista\n")
        f.write(f"Funzioni utilizzate: parse_chart_hits_list(), get_country_name(), safe_str()\n")
        f.write(f"File di output: foreign_popularity.txt\n")
        
    else:
        f.write("⚠️ Nessun artista trovato con successo in paesi esteri\n")
        f.write("Verifica i dati chart_hits nel dataset\n")

print(f"\n✅ File salvato con successo: foreign_popularity.txt")
print(f"   Dimensione: {len(artist_stats)} artisti analizzati")
print(f"   Tutti i calcoli e i dati sono stati salvati nel file")

# Mostra un riepilogo nel notebook
print("\n" + "="*60)
print("RIEPILOGO STATISTICHE")
print("="*60)

if len(artist_stats) > 0:
    print(f"\n🎯 ARTISTI CON SUCCESSO ALL'ESTERO:")
    print(f"   • Totale: {len(artist_stats)} artisti")
    
    # Distribuzione per numero di paesi
    countries_dist = {}
    for artist in artist_stats:
        num_countries = artist['num_foreign_countries']
        countries_dist[num_countries] = countries_dist.get(num_countries, 0) + 1
    
    print(f"\n📊 DISTRIBUZIONE PER NUMERO DI PAESI ESTERI:")
    for num_countries in sorted(countries_dist.keys(), reverse=True):
        count = countries_dist[num_countries]
        pct = 100 * count / len(artist_stats)
        print(f"   • {num_countries} paesi: {count} artisti ({pct:.1f}%)")
    
    # Top 3 nazionalità
    nat_counts = {}
    for artist in artist_stats:
        nat = artist['nationality_full']
        nat_counts[nat] = nat_counts.get(nat, 0) + 1
    
    print(f"\n🌍 TOP 5 NAZIONALITÀ PIÙ INTERNAZIONALI:")
    for nat, count in sorted(nat_counts.items(), key=lambda x: x[1], reverse=True)[:5]:
        pct = 100 * count / len(artist_stats)
        print(f"   • {nat}: {count} artisti ({pct:.1f}%)")
    
    # Artista con più paesi
    max_countries = max(artist_stats, key=lambda x: x['num_foreign_countries'])
    print(f"\n🏆 ARTISTA RECORD:")
    print(f"   • Più paesi: {max_countries['name']} ({max_countries['num_foreign_countries']} paesi)")
    
    # Artista più popolare
    max_popularity = max(artist_stats, key=lambda x: x['popularity'])
    print(f"   • Più popolare: {max_popularity['name']} (popolarità: {max_popularity['popularity']:.0f})")